In [37]:
from dataclasses import dataclass

import pandas as pd
import numpy as np

import prody
from biopandas.pdb import PandasPdb
from biotite.sequence import ProteinSequence


from predictability.constants import DATA_ROOT
from predictability.utils import (
    read_fasta,
    distance,
    dist_to_active_site,
    get_buriedness,
    assign_mutations,
    assign_classes,
)

In [38]:
data_dir = DATA_ROOT / "erk2"
data_dir.mkdir(exist_ok=True)
pdb_id = "3sa0"

In [39]:
filename = data_dir / f"{pdb_id}.pdb"
structure = PandasPdb().read_pdb(str(data_dir / f"{pdb_id}.pdb")).df["ATOM"].query('alt_loc == "" or alt_loc == "A"')
residue_characteristics = (
    structure[["residue_number", "residue_name"]]
    .drop_duplicates()
    .assign(
    residue_name=lambda d: d.residue_name.apply(ProteinSequence.convert_letter_3to1)
))

In [40]:
reference_sequence = [value for _, value in read_fasta(data_dir / "reference.fasta").items()][0]

# Calculating buriedness

In [41]:
buriedness = get_buriedness(filename).loc[
    lambda d: (d.chain_id == "A") & (~d.residue_name.isin(["HOH", "ACI", "CA"]))
]

residue_characteristics = residue_characteristics.merge(buriedness[["residue_number", "buriedness"]], on=["residue_number"])

# Calculating number of contacts

In [44]:
ca_atoms = (
    structure.loc[lambda d: d.atom_name == "CA"] .copy()
    .assign(contacts=np.nan)[
            ["residue_number", "contacts", "x_coord", "y_coord", "z_coord"]
        ].set_index("residue_number")
)
for ca in ca_atoms.itertuples():
    contacts = 0
    for other in ca_atoms.itertuples():
        if ca.Index != other.Index:
            dist = distance(ca, other)
            if dist < 7.3:
                contacts += 1
    ca_atoms.loc[ca.Index, "contacts"] = contacts
residue_characteristics = residue_characteristics.merge(ca_atoms.reset_index()[["residue_number", "contacts"]])

# Calculating distance to the active site

In [47]:
active_site_residues = [149]
active_site = list(structure.loc[lambda d: (d.residue_number.isin(active_site_residues)) & (d.atom_name == "CA")].itertuples())
as_distances = (
    structure.loc[lambda d: d.atom_name == "CA"] .copy()
    [
        ["residue_number", "x_coord", "y_coord", "z_coord"]
    ].set_index("residue_number")
)
as_distances["distance_to_active_site"] = [dist_to_active_site(ca, active_site) for ca in as_distances.itertuples()]
residue_characteristics = residue_characteristics.merge(as_distances.reset_index()[["residue_number", "distance_to_active_site"]])

# Determining secondary structure

In [49]:
_, header = prody.parsePDB(str(data_dir / f"{pdb_id}.pdb"), header=True)
residues = structure.loc[lambda d: d.atom_name == "CA"].copy()[["residue_number"]]

@dataclass
class Range:
    start: int
    stop: int

    def __contains__(self, value):
        return self.stop >= value >= self.start

ranges = [Range(start, stop) for _, _, _, _, start, stop in header["helix_range"] + header["sheet_range"]]

residues["is_secondary"] = [any(v in r for r in ranges) for v in structure.residue_number.unique()]
residue_characteristics = residue_characteristics.merge(residues)

@> 6115 atoms and 1 coordinate set(s) were parsed in 0.04s.


In [50]:
# Binarize
residue_characteristics["is_buried"] = (residue_characteristics.buriedness > residue_characteristics.buriedness.quantile(1 - 1/2)).astype(bool)
residue_characteristics["is_connected"] = (residue_characteristics.contacts > residue_characteristics.contacts.quantile(1 - 1/2)).astype(bool)
residue_characteristics["is_close_to_as"] = (residue_characteristics.distance_to_active_site < residue_characteristics.distance_to_active_site.quantile(1 - 1/2)).astype(bool)

In [51]:
# Save
residue_characteristics.to_csv(data_dir / "structural_characteristics.csv", index=False)

In [52]:
residue_characteristics

,residue_number,residue_name,buriedness,contacts,distance_to_active_site,is_secondary,is_buried,is_connected,is_close_to_as
0,4,A,0.760690,4.0,36.047513,False,False,False,False
1,5,A,1.409164,7.0,34.326936,False,False,False,False
2,6,A,0.613822,5.0,36.513661,False,False,False,False
3,7,A,2.836883,7.0,33.552993,False,False,False,False
4,8,G,1.973472,9.0,31.955743,False,False,True,False
...,...,...,...,...,...,...,...,...,...
349,356,P,7.513598,5.0,34.042062,False,True,False,False
350,357,G,4.776345,4.0,35.077865,False,False,False,False
351,358,Y,2.759526,4.0,38.032683,False,False,False,False
352,359,R,4.090262,4.0,40.058357,False,False,False,False


# Loading data

In [53]:
data = pd.read_csv(data_dir / "data_from_protein_gym.csv")
data["sequence"] = data["mutated_sequence"]
data = assign_mutations(data, str(reference_sequence))
feature_table = pd.read_csv(data_dir / "structural_characteristics.csv")
data = assign_classes(data, feature_table, mutation_col="mutations", features=["is_buried", "is_connected", "is_close_to_as", "is_secondary"])
data.to_csv(data_dir / "singles.csv")

In [54]:
data

,mutant,mutated_sequence,DMS_score,DMS_score_bin,sequence,mutations,residue_number,is_buried,is_connected,is_close_to_as,is_secondary
0,A2D,MDAAAAAGAGPEMVRGQVFDVGPRYTNLSYIGEGAYGMVCSAYDNV...,-7.840212,1,MDAAAAAGAGPEMVRGQVFDVGPRYTNLSYIGEGAYGMVCSAYDNV...,A002D,2,NaN,NaN,NaN,NaN
1,A2Y,MYAAAAAGAGPEMVRGQVFDVGPRYTNLSYIGEGAYGMVCSAYDNV...,-7.178781,1,MYAAAAAGAGPEMVRGQVFDVGPRYTNLSYIGEGAYGMVCSAYDNV...,A002Y,2,NaN,NaN,NaN,NaN
2,A2W,MWAAAAAGAGPEMVRGQVFDVGPRYTNLSYIGEGAYGMVCSAYDNV...,-5.469024,1,MWAAAAAGAGPEMVRGQVFDVGPRYTNLSYIGEGAYGMVCSAYDNV...,A002W,2,NaN,NaN,NaN,NaN
3,A2V,MVAAAAAGAGPEMVRGQVFDVGPRYTNLSYIGEGAYGMVCSAYDNV...,-6.268234,1,MVAAAAAGAGPEMVRGQVFDVGPRYTNLSYIGEGAYGMVCSAYDNV...,A002V,2,NaN,NaN,NaN,NaN
4,A2T,MTAAAAAGAGPEMVRGQVFDVGPRYTNLSYIGEGAYGMVCSAYDNV...,-9.346828,0,MTAAAAAGAGPEMVRGQVFDVGPRYTNLSYIGEGAYGMVCSAYDNV...,A002T,2,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
6804,S360C,MAAAAAAGAGPEMVRGQVFDVGPRYTNLSYIGEGAYGMVCSAYDNV...,-7.836966,1,MAAAAAAGAGPEMVRGQVFDVGPRYTNLSYIGEGAYGMVCSAYDNV...,S360C,360,False,False,False,False
6805,S360A,MAAAAAAGAGPEMVRGQVFDVGPRYTNLSYIGEGAYGMVCSAYDNV...,-9.465173,0,MAAAAAAGAGPEMVRGQVFDVGPRYTNLSYIGEGAYGMVCSAYDNV...,S360A,360,False,False,False,False
6806,S360W,MAAAAAAGAGPEMVRGQVFDVGPRYTNLSYIGEGAYGMVCSAYDNV...,-7.744855,1,MAAAAAAGAGPEMVRGQVFDVGPRYTNLSYIGEGAYGMVCSAYDNV...,S360W,360,False,False,False,False
6807,S360H,MAAAAAAGAGPEMVRGQVFDVGPRYTNLSYIGEGAYGMVCSAYDNV...,-8.199658,0,MAAAAAAGAGPEMVRGQVFDVGPRYTNLSYIGEGAYGMVCSAYDNV...,S360H,360,False,False,False,False
